# Notebook 08: Phase 4 - Dự báo đệ quy (Recursive Forecasting)

**Mục tiêu:**
- Thực hiện **Dự báo đệ quy (Recursive Forecasting)** từ 2015 đến 2019.
- **Yêu cầu (User Request)**: 
    1. **Global Baseline**: Sử dụng `data/processed/lr_final_prep.csv` (Dữ liệu đã StandardScaled + OHE).
    2. **Hybrid Model**: Sử dụng `data/processed/common_preprocessed.csv` (Dữ liệu gốc), thực hiện tiền xử lý và training **ON-THE-FLY** (giống Notebook 07).
- **Cơ chế**: 
    - Duy trì 2 luồng dữ liệu song song trong vòng lặp đệ quy.
    - **Global**: Update Lag đã Scaled.
    - **Hybrid**: Update Lag Raw (hoặc theo quy trình của Hybrid).
- **Đánh giá**: So sánh sai số tích lũy.

In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import os
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# Constants
SPLIT_YEAR = 2015
TEST_END_YEAR = 2019
TARGET = 'Value_co2_emissions_kt_by_country'
LAG_TARGET = f'{TARGET}_lag1'

DATA_DIR = '../data/processed/'
MODEL_DIR = '../models/'
RESULTS_DIR = '../data/results/'

os.makedirs(RESULTS_DIR, exist_ok=True)

## 1. Chuẩn bị Dữ liệu (Dual Data Streams)

Chúng ta cần load 2 bộ dữ liệu riêng biệt:
1. `df_lr`: Cho Linear Regression (đã Scaled).
2. `df_common`: Cho Hybrid Model (Raw).

In [ ]:
def load_dual_data():
    # 1. LOAD GLOBAL DATA (LR Final Prep)
    print("Loading Global Data (lr_final_prep.csv)...")
    df_lr = pd.read_csv(os.path.join(DATA_DIR, 'lr_final_prep.csv'))
    if 'Year' in df_lr.columns:
        df_lr['Year'] = pd.to_numeric(df_lr['Year'])
        
    # Recover Entity for mapping if missing (from OHE)
    if 'Entity' not in df_lr.columns:
        print("Recovering 'Entity' from OHE in df_lr...")
        entity_cols = [c for c in df_lr.columns if c.startswith('Entity_')]
        df_lr['Entity'] = df_lr[entity_cols].idxmax(axis=1).apply(lambda x: x.replace('Entity_', ''))

    # 2. LOAD HYBRID DATA (Common Preprocessed)
    print("Loading Hybrid Data (common_preprocessed.csv)...")
    df_common = pd.read_csv(os.path.join(DATA_DIR, 'common_preprocessed.csv'))
    if 'Year' in df_common.columns:
        df_common['Year'] = pd.to_numeric(df_common['Year'])
        
    # Preprocessing for Hybrid (as per Notebook 07 LOGIC)
    # Notebook 07 trained on lr_final_prep actually, but user wants us to use COMMON here.
    # Common has incomplete features maybe? 
    # Let's generate OHE for Entity in Common because Hybrid Backbone (Ridge) needs it.
    print("Preprocessing Grid for Hybrid (OHE)...")
    dummies = pd.get_dummies(df_common['Entity'], prefix='Entity', dtype=int)
    df_hybrid = pd.concat([df_common, dummies], axis=1)
    
    # Hybrid in 07 used 'lr_final_prep' which was Scaled.
    # If we use Common (Raw), we might need to Scale it to be comparable or strictly follow user.
    # User said: "hybrid thì dùng common (nói chung là trước đấy huấn luyện sao giờ huấn luyện lại y chang)"
    # If 07 used Scaled data, and we must use Common (Raw), then we SHOULD Scale Common.
    # Let's Scale df_hybrid numeric features.
    print("Scaling Hybrid Data (StandardScaler logic)...")
    # Exclude Entity, Year, Target, OHE columns (0/1)
    non_scale_cols = [TARGET, 'Year', 'Entity'] + [c for c in df_hybrid.columns if c.startswith('Entity_')]
    scale_cols = [c for c in df_hybrid.columns if c not in non_scale_cols]
    
    # Fit scaler on TRAIN ONLY (< 2015)
    train_mask = df_hybrid['Year'] < SPLIT_YEAR
    means = df_hybrid.loc[train_mask, scale_cols].mean()
    stds = df_hybrid.loc[train_mask, scale_cols].std().replace(0, 1)
    
    for col in scale_cols:
        df_hybrid[col] = (df_hybrid[col] - means[col]) / stds[col]
        
    # Store stats for recursive update later
    hybrid_stats = {'means': means, 'stds': stds}
    
    return df_lr, df_hybrid, hybrid_stats

df_lr, df_hybrid, hybrid_stats = load_dual_data()
print(f"Global Data (Scaled): {df_lr.shape}")
print(f"Hybrid Data (Scaled form Common): {df_hybrid.shape}")

## 2. Train Models ON-THE-FLY

Huấn luyện lại cả 2 mô hình trên tập Training (< 2015) của dữ liệu tương ứng.

In [ ]:
# Define Features
drop_cols_lr = [TARGET, 'Year', 'Entity'] 
features_lr = [c for c in df_lr.columns if c not in drop_cols_lr]

drop_cols_hybrid = [TARGET, 'Year', 'Entity']
features_hybrid = [c for c in df_hybrid.columns if c not in drop_cols_hybrid]

# SPLIT TRAIN (< 2015)
mask_train_lr = df_lr['Year'] < SPLIT_YEAR
X_train_lr = df_lr.loc[mask_train_lr, features_lr]
y_train_lr = df_lr.loc[mask_train_lr, TARGET]

mask_train_hb = df_hybrid['Year'] < SPLIT_YEAR
X_train_hb = df_hybrid.loc[mask_train_hb, features_hybrid]
y_train_hb = df_hybrid.loc[mask_train_hb, TARGET]

# --- 1. Train Global Baseline (Ridge) ---
print("Training Global Baseline (Ridge)...")
model_global = Ridge(alpha=10.0)
model_global.fit(X_train_lr, y_train_lr)
print("Global Baseline Trained.")

# --- 2. Train Hybrid (Ridge + XGB) ---
print("Training Hybrid Model (Ridge + XGB)...")
# A. Backbone
hybrid_ridge = Ridge(alpha=10.0)
hybrid_ridge.fit(X_train_hb, y_train_hb)
y_pred_backbone = hybrid_ridge.predict(X_train_hb)
residuals_train = y_train_hb - y_pred_backbone

# B. Residual Corrector (XGBoost)
xgb_params = {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.05, 'n_jobs': -1, 'random_state': 42}
hybrid_xgb = XGBRegressor(**xgb_params)
hybrid_xgb.fit(X_train_hb, residuals_train)
print("Hybrid Model Trained.")

## 3. Recursive Forecasting Loop (Dual Stream)

Chúng ta xử lý từng năm:
- Dự báo cho năm `t`.
- Cập nhật Lag cho năm `t+1`.
- **Lưu ý**: 
    - `df_lr` cần update Lag theo kiểu Scaled (Dùng Stats của df_lr).
    - `df_hybrid` cần update Lag theo kiểu Scaled (Dùng Stats của df_common mà ta đã tính `hybrid_stats`).
    - Target ở cả 2 dataframe đều là Raw Value (kt CO2), nên logic (Pred - Mean)/Std là giống nhau, chỉ khác giá trị Mean/Std của từng bộ dữ liệu.

In [ ]:
print("Bắt đầu Dự báo Đệ quy (2015-2019)...")

# Calculate Target stats for Scaling Lag (Raw Target -> Scaled Lag)
# NB: We need stats from the respective training sets
target_mean_lr = y_train_lr.mean()
target_std_lr = y_train_lr.std()

target_mean_hb = y_train_hb.mean()
target_std_hb = y_train_hb.std()

# Prepare Test Copies
df_test_lr = df_lr[df_lr['Year'] >= SPLIT_YEAR].copy()
df_test_hb = df_hybrid[df_hybrid['Year'] >= SPLIT_YEAR].copy()

predictions_global = []
predictions_hybrid = []

years_to_predict = sorted(df_test_lr['Year'].unique())

for year in years_to_predict:
    print(f"\n>>> Processing Year: {year}")
    
    # --- A. GLOBAL (df_lr stream) ---
    current_year_lr = df_test_lr[df_test_lr['Year'] == year]
    X_curr_lr = current_year_lr[features_lr]
    
    y_pred_g = model_global.predict(X_curr_lr)
    y_pred_g = np.maximum(y_pred_g, 0)
    
    res_g = current_year_lr[['Entity', 'Year', TARGET]].copy()
    res_g['Pred_Global'] = y_pred_g
    predictions_global.append(res_g)
    
    # Update Next Lag (Global)
    next_year = year + 1
    if next_year <= TEST_END_YEAR:
        pred_map = dict(zip(current_year_lr['Entity'], y_pred_g))
        # Scale: (Pred - Mean) / Std
        pred_map_scaled = {k: (v - target_mean_lr) / target_std_lr for k, v in pred_map.items()}
        
        mask_next = df_test_lr['Year'] == next_year
        next_ents = df_test_lr.loc[mask_next, 'Entity']
        # Assuming lag col name is same
        updated_lags = next_ents.map(pred_map_scaled).fillna(df_test_lr.loc[mask_next, LAG_TARGET])
        df_test_lr.loc[mask_next, LAG_TARGET] = updated_lags

    # --- B. HYBRID (df_hybrid stream) ---
    current_year_hb = df_test_hb[df_test_hb['Year'] == year]
    X_curr_hb = current_year_hb[features_hybrid]
    
    pred_ridge = hybrid_ridge.predict(X_curr_hb)
    pred_xgb = hybrid_xgb.predict(X_curr_hb)
    y_pred_h = pred_ridge + pred_xgb
    y_pred_h = np.maximum(y_pred_h, 0)
    
    res_h = current_year_hb[['Entity', 'Year', TARGET]].copy()
    res_h['Pred_Hybrid'] = y_pred_h
    predictions_hybrid.append(res_h)
    
    # Update Next Lag (Hybrid)
    if next_year <= TEST_END_YEAR:
        pred_map_h = dict(zip(current_year_hb['Entity'], y_pred_h))
        # Scale using Hybrid Data stats
        pred_map_h_scaled = {k: (v - target_mean_hb) / target_std_hb for k, v in pred_map_h.items()}
        
        mask_next_h = df_test_hb['Year'] == next_year
        next_ents_h = df_test_hb.loc[mask_next_h, 'Entity']
        updated_lags_h = next_ents_h.map(pred_map_h_scaled).fillna(df_test_hb.loc[mask_next_h, LAG_TARGET])
        df_test_hb.loc[mask_next_h, LAG_TARGET] = updated_lags_h

# Merge Results
df_res_global = pd.concat(predictions_global)
df_res_hybrid = pd.concat(predictions_hybrid)
final_comparison = pd.merge(
    df_res_global[['Entity', 'Year', TARGET, 'Pred_Global']],
    df_res_hybrid[['Entity', 'Year', 'Pred_Hybrid']],
    on=['Entity', 'Year']
)
print("\nHoàn tất forecast.")

## 4. Evaluation & Visualization

In [ ]:
def calculate_metrics(df, true_col, pred_col, label):
    # Store APE in specific column
    true_safe = df[true_col].replace(0, 1e-6)
    df[f'APE_{label}'] = np.abs((df[true_col] - df[pred_col]) / true_safe) * 100
    
    mse = mean_squared_error(df[true_col], df[pred_col])
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(df[true_col], df[pred_col])
    r2 = r2_score(df[true_col], df[pred_col])
    median_mape = df.groupby('Entity')[f'APE_{label}'].mean().median()
    
    return rmse, mae, r2, median_mape

rmse_g, mae_g, r2_g, mape_g = calculate_metrics(final_comparison, TARGET, 'Pred_Global', 'Global')
rmse_h, mae_h, r2_h, mape_h = calculate_metrics(final_comparison, TARGET, 'Pred_Hybrid', 'Hybrid')

print("=== KẾT QUẢ ĐÁNH GIÁ (RECURSIVE 2015-2019) ===")
print(f"{'Metric':<15} | {'Global Model':<15} | {'Hybrid Model':<15} | {'Cải thiện':<10}")
print("-"*70)
print(f"{'RMSE':<15} | {rmse_g:,.2f}        | {rmse_h:,.2f}         | {(rmse_g - rmse_h)/rmse_g*100:+.2f}%")
print(f"{'MAE':<15} | {mae_g:,.2f}        | {mae_h:,.2f}         | {(mae_g - mae_h)/mae_g*100:+.2f}%")
print(f"{'Median MAPE':<15} | {mape_g:.2f}%          | {mape_h:.2f}%           | {(mape_g - mape_h)/mape_g*100:+.2f}%")
print(f"{'R2 Score':<15} | {r2_g:.4f}          | {r2_h:.4f}           | {(r2_h - r2_g):+.4f}")

# 5.1 Global Trend
global_trend = final_comparison.groupby('Year')[[TARGET, 'Pred_Global', 'Pred_Hybrid']].sum().reset_index()
plt.figure(figsize=(10, 6))
plt.plot(global_trend['Year'], global_trend[TARGET], label='Actual', marker='o', linewidth=2, color='black')
plt.plot(global_trend['Year'], global_trend['Pred_Global'], label='Global Baseline', marker='s', linestyle='--', color='blue')
plt.plot(global_trend['Year'], global_trend['Pred_Hybrid'], label='Hybrid Model', marker='^', linestyle='--', color='red')
plt.title('Global Recursive Forecasting (2015-2019)', fontsize=16)
plt.xlabel('Year')
plt.ylabel('Total CO2 Emissions (kt)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 5.2 Top Countries
countries = ['China', 'United States', 'India', 'Vietnam']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
for i, country in enumerate(countries):
    country_data = final_comparison[final_comparison['Entity'] == country]
    if len(country_data) > 0:
        ax = axes[i]
        ax.plot(country_data['Year'], country_data[TARGET], label='Actual', marker='o', color='black')
        ax.plot(country_data['Year'], country_data['Pred_Global'], label='Global Baseline', linestyle='--', color='blue')
        ax.plot(country_data['Year'], country_data['Pred_Hybrid'], label='Hybrid Model', linestyle='--', color='red')
        ax.set_title(f'Recursive Forecast: {country}')
        ax.legend()
plt.tight_layout()
plt.show()

# 5.3 Error Dist
ape_data = final_comparison.melt(id_vars=['Entity', 'Year'], value_vars=['APE_Global', 'APE_Hybrid'], var_name='Model', value_name='APE')
plt.figure(figsize=(10, 6))
sns.boxplot(data=ape_data, x='Model', y='APE', showfliers=False)
plt.title('APE Distribution', fontsize=16)
plt.show()